In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
glcm = pd.read_csv('C:/Users/ravee/Jupyter/Skin-Cancer-Classification/data/glcm3.csv', header=None)
lbp = pd.read_csv('C:/Users/ravee/Jupyter/Skin-Cancer-Classification/data/lbp3.csv', header=None)
dataset = pd.concat([lbp, glcm], axis = 1, ignore_index = True)
X = dataset.iloc[:, 0:50].values
Y = dataset.iloc[:, 50].values

In [3]:
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [4]:
len(X_train)

736

In [5]:
len(X_test)

185

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Naive Bayes

In [7]:
# Implementation of Naive Bayes
# Fitting Naive Bayes to the training set

from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [8]:
# Predicting the test set result
y_pred = nb.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[60, 10,  9],
       [28, 21, 11],
       [27,  7, 12]], dtype=int64)

In [9]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = nb, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.5272306553128472


In [10]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = nb, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.54      0.75      0.63        40
           2       0.38      0.19      0.25        32
           3       0.43      0.43      0.43        21

    accuracy                           0.48        93
   macro avg       0.45      0.46      0.43        93
weighted avg       0.46      0.48      0.45        93

              precision    recall  f1-score   support

           1       0.60      0.82      0.70        39
           2       0.75      0.55      0.63        33
           3       0.33      0.25      0.29        20

    accuracy                           0.60        92
   macro avg       0.56      0.54      0.54        92
weighted avg       0.60      0.60      0.58        92

              precision    recall  f1-score   support

           1       0.57      0.79      0.67        39
           2       0.72      0.39      0.51        33
           3       0.35      0.35      0.35        20

    accuracy        

In [11]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.57      0.75      0.65       391
         bcc       0.59      0.39      0.47       323
         scc       0.31      0.28      0.30       207

    accuracy                           0.52       921
   macro avg       0.49      0.47      0.47       921
weighted avg       0.52      0.52      0.50       921



# K-Nearest Neighbor

In [12]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 23, metric = 'minkowski', p = 3, weights = 'distance')
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=23, p=3,
                     weights='distance')

In [13]:
# Predicting the test set result
y_pred = knn.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[66, 10,  3],
       [22, 33,  5],
       [16, 19, 11]], dtype=int64)

In [14]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = knn, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.6534801925212884


In [15]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'n_neighbors': [21, 23, 25, 27, 29], 'weights': ['uniform', 'distance'], 'p': [1, 2, 3], 'metric': ['minkowski']},
]

grid_search = GridSearchCV(estimator = knn, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.65
{'metric': 'minkowski', 'n_neighbors': 23, 'p': 3, 'weights': 'distance'}


In [16]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = knn, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.58      0.65      0.61        40
           2       0.44      0.56      0.49        32
           3       0.29      0.10      0.14        21

    accuracy                           0.49        93
   macro avg       0.43      0.44      0.42        93
weighted avg       0.46      0.49      0.47        93

              precision    recall  f1-score   support

           1       0.65      0.79      0.71        39
           2       0.66      0.76      0.70        33
           3       0.50      0.15      0.23        20

    accuracy                           0.64        92
   macro avg       0.60      0.57      0.55        92
weighted avg       0.62      0.64      0.60        92

              precision    recall  f1-score   support

           1       0.66      0.85      0.74        39
           2       0.67      0.67      0.67        33
           3       0.78      0.35      0.48        20

    accuracy        

In [17]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.61      0.72      0.66       391
         bcc       0.57      0.62      0.59       323
         scc       0.44      0.22      0.29       207

    accuracy                           0.57       921
   macro avg       0.54      0.52      0.51       921
weighted avg       0.56      0.57      0.55       921



# Support Vector Machine

In [36]:
# Implementation of Support Vector Machine (SVM)
# Fitting SVM to the training set

from sklearn.svm import SVC
svm = SVC(C = 10, kernel = 'rbf', gamma = 0.01, random_state = 0)
#classifier = SVC(C = 1, kernel = 'poly', degree = 30, random_state = 0)
svm.fit(X_train, y_train)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [37]:
# Predicting the test set result
y_pred = svm.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[71,  7,  1],
       [17, 37,  6],
       [ 5, 21, 20]], dtype=int64)

In [38]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = svm, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.6915216586449462


In [31]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'C': [1, 10, 100], 'kernel': ['rbf', 'sigmoid', 'poly'], 'gamma': [0.001, 0.01, 0.1, 10]},
]

grid_search = GridSearchCV(estimator = svm, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.69
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [45]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred, zero_division = 0))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = svm, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.45      0.85      0.59        40
           2       0.35      0.19      0.24        32
           3       1.00      0.00      0.00        21

    accuracy                           0.43        93
   macro avg       0.60      0.35      0.28        93
weighted avg       0.54      0.43      0.34        93

              precision    recall  f1-score   support

           1       0.51      0.95      0.67        39
           2       0.85      0.52      0.64        33
           3       1.00      0.00      0.00        20

    accuracy                           0.59        92
   macro avg       0.79      0.49      0.44        92
weighted avg       0.74      0.59      0.51        92

              precision    recall  f1-score   support

           1       0.51      0.95      0.66        39
           2       0.68      0.39      0.50        33
           3       1.00      0.00      0.00        20

    accuracy        

In [48]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names, zero_division = 0))

              precision    recall  f1-score   support

    melanoma       0.49      0.90      0.63       391
         bcc       0.60      0.36      0.45       323
         scc       0.00      0.00      0.00       207

    accuracy                           0.51       921
   macro avg       0.36      0.42      0.36       921
weighted avg       0.42      0.51      0.43       921



# Decision Tree

In [49]:
# Some important notes here, before running the Decision Tree Classifier, it's ok to not scale the features, as this model
# does not work using euclidian distance, so it's fine as it is

from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion = 'gini', max_depth = 20, random_state = 0, splitter = 'best')
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=20, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [50]:
# Predicting the test set result
y_pred = tree.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[54, 16,  9],
       [12, 34, 14],
       [10, 18, 18]], dtype=int64)

In [51]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = tree, X = X_train, y = y_train, cv = 10)
print(np.mean(accuracies))

0.5584042947056644


In [11]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'criterion': ['entropy', 'gini'], 'splitter': ['best', 'random'], 'ccp_alpha': [0.0, 0.05, 0.1], 'max_depth': [10, 20, 30]}
]

grid_search = GridSearchCV(estimator = tree, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.56
{'ccp_alpha': 0.0, 'criterion': 'gini', 'max_depth': 20, 'splitter': 'best'}


In [52]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = tree, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.76      0.70      0.73        40
           2       0.40      0.38      0.39        32
           3       0.23      0.29      0.26        21

    accuracy                           0.49        93
   macro avg       0.46      0.45      0.46        93
weighted avg       0.52      0.49      0.50        93

              precision    recall  f1-score   support

           1       0.78      0.72      0.75        39
           2       0.55      0.52      0.53        33
           3       0.24      0.30      0.27        20

    accuracy                           0.55        92
   macro avg       0.52      0.51      0.51        92
weighted avg       0.58      0.55      0.57        92

              precision    recall  f1-score   support

           1       0.74      0.67      0.70        39
           2       0.53      0.64      0.58        33
           3       0.41      0.35      0.38        20

    accuracy        

In [53]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.70      0.68      0.69       391
         bcc       0.51      0.50      0.51       323
         scc       0.32      0.34      0.33       207

    accuracy                           0.54       921
   macro avg       0.51      0.51      0.51       921
weighted avg       0.55      0.54      0.54       921



# Random Forest

In [54]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 250, max_depth = None, criterion = 'entropy', class_weight = 'balanced_subsample', random_state = 0)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight='balanced_subsample', criterion='entropy',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=250, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)

In [55]:
# Predicting the test set result
y_pred = forest.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[66, 11,  2],
       [10, 45,  5],
       [10, 20, 16]], dtype=int64)

In [56]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = forest, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(np.mean(accuracies))

0.6657904479822288


In [10]:
# Applying Grid Search to Find the Best Hyperparameter

from sklearn.model_selection import GridSearchCV
parameters = [
    {'criterion': ['entropy'], 'max_depth': [None], 'n_estimators': [150, 200, 250], 'class_weight': [None, 'balanced', 'balanced_subsample']},
    {'criterion': ['gini'], 'max_depth': [None], 'n_estimators': [150, 200, 250], 'class_weight': [None, 'balanced', 'balanced_subsample']}
]

grid_search = GridSearchCV(estimator = forest, param_grid = parameters, scoring = 'accuracy', cv = 10, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.67
{'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': None, 'n_estimators': 250}


In [57]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = forest, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.83      0.85      0.84        40
           2       0.56      0.75      0.64        32
           3       0.44      0.19      0.27        21

    accuracy                           0.67        93
   macro avg       0.61      0.60      0.58        93
weighted avg       0.65      0.67      0.64        93

              precision    recall  f1-score   support

           1       0.81      0.87      0.84        39
           2       0.72      0.85      0.78        33
           3       0.73      0.40      0.52        20

    accuracy                           0.76        92
   macro avg       0.75      0.71      0.71        92
weighted avg       0.76      0.76      0.75        92

              precision    recall  f1-score   support

           1       0.76      0.82      0.79        39
           2       0.68      0.85      0.76        33
           3       0.44      0.20      0.28        20

    accuracy        

In [58]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.75      0.81      0.78       391
         bcc       0.63      0.75      0.68       323
         scc       0.56      0.31      0.40       207

    accuracy                           0.68       921
   macro avg       0.65      0.62      0.62       921
weighted avg       0.66      0.68      0.66       921



# Artificial Neural Network

In [64]:
# Importing the Keras libraries and packages
import keras
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [66]:
# Create categorical input for predictor

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore')
Y = Y.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_use = onehotencoder.fit_transform(Y).toarray()
y_binary = onehotencoder.fit_transform(y_train).toarray()
y_compare = onehotencoder.fit_transform(y_test).toarray()

In [67]:
# Initializing the ANN
ann = Sequential()

# Adding the input & hidden layer
ann.add(Dense(32, activation = 'relu', input_shape = (50,)))
ann.add(Dense(32, activation = 'relu'))
ann.add(Dense(32, activation = 'relu'))
ann.add(Dense(3, activation = 'softmax'))
ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

ann.fit(X_train, y_binary, batch_size = 10, epochs = 100)

Epoch 1/100
736/736 [==============================] - 3s 4ms/step - loss: 1.0621 - accuracy: 0.4620
Epoch 2/100
736/736 [==============================] - 0s 91us/step - loss: 0.9071 - accuracy: 0.5856
Epoch 3/100
736/736 [==============================] - 0s 86us/step - loss: 0.8338 - accuracy: 0.6318
Epoch 4/100
736/736 [==============================] - 0s 84us/step - loss: 0.7823 - accuracy: 0.6495
Epoch 5/100
736/736 [==============================] - 0s 97us/step - loss: 0.7473 - accuracy: 0.6685
Epoch 6/100
736/736 [==============================] - 0s 93us/step - loss: 0.7186 - accuracy: 0.6861
Epoch 7/100
736/736 [==============================] - 0s 88us/step - loss: 0.6862 - accuracy: 0.6902
Epoch 8/100
736/736 [==============================] - 0s 89us/step - loss: 0.6763 - accuracy: 0.6970
Epoch 9/100
736/736 [==============================] - 0s 88us/step - loss: 0.6581 - accuracy: 0.7065
Epoch 10/100
736/736 [==============================] - 0s 89us/step - loss: 0.6336

736/736 [==============================] - 0s 88us/step - loss: 0.0989 - accuracy: 0.9783
Epoch 80/100
736/736 [==============================] - 0s 88us/step - loss: 0.0839 - accuracy: 0.9837
Epoch 81/100
736/736 [==============================] - 0s 87us/step - loss: 0.0861 - accuracy: 0.9823
Epoch 82/100
736/736 [==============================] - 0s 93us/step - loss: 0.0841 - accuracy: 0.9851
Epoch 83/100
736/736 [==============================] - 0s 107us/step - loss: 0.0924 - accuracy: 0.9755
Epoch 84/100
736/736 [==============================] - 0s 91us/step - loss: 0.1296 - accuracy: 0.9565
Epoch 85/100
736/736 [==============================] - 0s 88us/step - loss: 0.1243 - accuracy: 0.9552
Epoch 86/100
736/736 [==============================] - 0s 87us/step - loss: 0.1700 - accuracy: 0.9511
Epoch 87/100
736/736 [==============================] - 0s 89us/step - loss: 0.0929 - accuracy: 0.9688
Epoch 88/100
736/736 [==============================] - 0s 86us/step - loss: 0.0773 -

In [68]:
# Create threshold for multi-predictor
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [69]:
# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_compare.argmax(axis=1), y_pred.argmax(axis=1))

cm

array([[70,  7,  2],
       [ 9, 37, 14],
       [ 6, 23, 17]], dtype=int64)

In [71]:
# Calculating the accuracy and other metrics
#, scoring=make_scorer(classification_report_with_accuracy_score)

from sklearn.model_selection import StratifiedKFold, cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Build the Model
def buildmodel():
    ann = Sequential()
    ann.add(Dense(16, activation = 'relu', input_shape = (50,)))
    ann.add(Dense(16, activation = 'relu'))
    ann.add(Dense(3, activation = 'softmax'))
    ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_accuracy'])
    return(ann)


#Make our customer score
def classification_report_with_accuracy_score(y_use, y_pred):
    originalclass.extend(y_use)
    predictedclass.extend(y_pred.argmax(axis=1))
    print(classification_report(y_use, y_pred.argmax(axis=1)))
    return accuracy_score(y_use, y_pred.argmax(axis=1)) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
ann = KerasRegressor(build_fn=buildmodel, epochs = 100, batch_size = 10, verbose = 0)
nested_score = cross_val_score(estimator = ann, X = X, y = y_use.argmax(1), cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.63      0.65      0.64        40
           1       0.42      0.69      0.52        32
           2       0.00      0.00      0.00        21

    accuracy                           0.52        93
   macro avg       0.35      0.45      0.39        93
weighted avg       0.42      0.52      0.46        93

              precision    recall  f1-score   support

           0       0.61      0.87      0.72        39
           1       0.76      0.76      0.76        33
           2       0.67      0.10      0.17        20

    accuracy                           0.66        92
   macro avg       0.68      0.58      0.55        92
weighted avg       0.67      0.66      0.61        92

              precision    recall  f1-score   support

           0       0.68      0.69      0.68        39
           1       0.65      0.79      0.71        33
           2       0.42      0.25      0.31        20

    accuracy        

C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.53      0.95      0.68        39
           1       0.64      0.42      0.51        33
           2       0.00      0.00      0.00        20

    accuracy                           0.55        92
   macro avg       0.39      0.46      0.40        92
weighted avg       0.45      0.55      0.47        92

              precision    recall  f1-score   support

           0       0.59      0.77      0.67        39
           1       0.55      0.69      0.61        32
           2       0.00      0.00      0.00        21

    accuracy                           0.57        92
   macro avg       0.38      0.49      0.43        92
weighted avg       0.44      0.57      0.50        92



C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.77      0.59      0.67        39
           1       0.47      0.91      0.62        32
           2       0.00      0.00      0.00        21

    accuracy                           0.57        92
   macro avg       0.41      0.50      0.43        92
weighted avg       0.49      0.57      0.50        92



C:\Users\ravee\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.60      0.82      0.70        39
           1       0.54      0.66      0.59        32
           2       0.00      0.00      0.00        21

    accuracy                           0.58        92
   macro avg       0.38      0.49      0.43        92
weighted avg       0.44      0.58      0.50        92

              precision    recall  f1-score   support

           0       0.54      0.67      0.60        39
           1       0.47      0.53      0.50        32
           2       0.25      0.10      0.14        21

    accuracy                           0.49        92
   macro avg       0.42      0.43      0.41        92
weighted avg       0.45      0.49      0.46        92

              precision    recall  f1-score   support

           0       0.54      0.79      0.65        39
           1       0.62      0.62      0.62        32
           2       0.33      0.05      0.08        21

    accuracy        

In [72]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.60      0.75      0.67       391
         bcc       0.55      0.68      0.61       323
         scc       0.40      0.07      0.12       207

    accuracy                           0.57       921
   macro avg       0.52      0.50      0.46       921
weighted avg       0.54      0.57      0.52       921



# XGBoost

In [59]:
# Importing the XGB Library and fit it to the data
from xgboost import XGBClassifier
xgb = XGBClassifier(objective = 'multi:softmax', booster = 'dart', n_estimators = 100, rate_drop = 0.1, max_depth = 3, normalize_type = 'forest', skip_drop = 0.1)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='dart', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              normalize_type='forest', nthread=None, objective='multi:softprob',
              random_state=0, rate_drop=0.1, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=None, silent=None, skip_drop=0.1,
              subsample=1, verbosity=1)

In [60]:
# Predicting the test set result
y_pred = xgb.predict(X_test)

# Making the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm2 = multilabel_confusion_matrix(y_test, y_pred)

cm

array([[67,  9,  3],
       [15, 40,  5],
       [13, 22, 11]], dtype=int64)

In [61]:
# Applying K-Fold Cross Validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = xgb, X = X_train, y = y_train, cv = 10, n_jobs = -1)
print(np.mean(accuracies))

0.6480377637911885


In [19]:
# Applying Grid Search to Find the Best Hyperparameter
# {'objective': ['multi:softmax'], 'n_estimators': [25, 50, 100], 'booster': ['gbtree'], 'eta': [0.001, 0.002, 0.004], 'max_depth':[3], 'gamma': [0], 'subsample': [1], 'sampling_method': ['uniform', 'gradient_based']},
# {'objective': ['multi:softmax'], 'n_estimators': [109], 'booster': ['dart'], 'sample_type': ['uniform', 'weighted'], 'normalize_type': ['tree', 'forest'], 'rate_drop': [0, 0.1, 0.2], 'skip_drop': [0, 0.1, 0.2]}
# {'objective': ['multi:softmax'], 'n_estimators': [10, 25, 50], 'booster': ['gblinear'], 'feature_selector': ['greedy', 'cyclic', 'random', 'shuffle', 'thrifty'], 'updater': ['shotgun', 'coord_descent'], 'lambda': [0, 0.5, 1], 'alpha': [0, 0.5, 1]}

from sklearn.model_selection import GridSearchCV
parameters = [
    {'objective': ['multi:softmax'], 'n_estimators': [25, 50, 100], 'booster': ['gbtree'], 'eta': [0.001, 0.002, 0.004], 'max_depth':[3], 'gamma': [0], 'subsample': [1], 'sampling_method': ['uniform', 'gradient_based']},
    {'objective': ['multi:softmax'], 'n_estimators': [25, 50, 100], 'booster': ['dart'], 'sample_type': ['uniform', 'weighted'], 'normalize_type': ['tree', 'forest'], 'rate_drop': [0, 0.1, 0.2], 'skip_drop': [0, 0.1, 0.2]}
]

grid_search = GridSearchCV(estimator = xgb, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print(round(best_accuracy, 2))
print(best_parameters)

0.68
{'booster': 'dart', 'n_estimators': 100, 'normalize_type': 'forest', 'objective': 'multi:softmax', 'rate_drop': 0.1, 'sample_type': 'uniform', 'skip_drop': 0.1}


In [62]:
# Calculating the accuracy and other metrics

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold

target_names = ['melanoma', 'bcc', 'scc']

# Variables for average classification report
originalclass = []
predictedclass = []

#Make our customer score
def classification_report_with_accuracy_score(y_test, y_pred):
    originalclass.extend(y_test)
    predictedclass.extend(y_pred)
    print(classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred) # return accuracy score

outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# Nested CV with parameter optimization
nested_score = cross_val_score(estimator = xgb, X = X, y = Y, cv=outer_cv, scoring=make_scorer(classification_report_with_accuracy_score))

              precision    recall  f1-score   support

           1       0.86      0.90      0.88        40
           2       0.52      0.69      0.59        32
           3       0.44      0.19      0.27        21

    accuracy                           0.67        93
   macro avg       0.61      0.59      0.58        93
weighted avg       0.65      0.67      0.64        93

              precision    recall  f1-score   support

           1       0.81      0.87      0.84        39
           2       0.75      0.73      0.74        33
           3       0.56      0.50      0.53        20

    accuracy                           0.74        92
   macro avg       0.71      0.70      0.70        92
weighted avg       0.73      0.74      0.74        92

              precision    recall  f1-score   support

           1       0.76      0.79      0.77        39
           2       0.64      0.82      0.72        33
           3       0.44      0.20      0.28        20

    accuracy        

In [63]:
# Average values in classification report for all folds in a K-fold Cross-validation  
print(classification_report(originalclass, predictedclass, target_names=target_names))

              precision    recall  f1-score   support

    melanoma       0.77      0.82      0.79       391
         bcc       0.61      0.71      0.66       323
         scc       0.46      0.28      0.35       207

    accuracy                           0.66       921
   macro avg       0.61      0.60      0.60       921
weighted avg       0.64      0.66      0.65       921

